In [ ]:
! pip install datasets nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 Small model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from datasets import load_dataset

# Load a sample of the BookCorpus dataset
dataset = load_dataset("bookcorpus", split='train[:5000000]')


In [ ]:
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 5000000
})


In [ ]:
import nltk

# Download the punkt tokenizer models
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from collections import Counter
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Assuming 'dataset' is already loaded and 'text' is the key containing your sentences
def count_stopwords(dataset):
    counter = Counter()
    for text in dataset['text']:
        words = nltk.word_tokenize(text.lower())  # Tokenize and convert to lower case
        filtered_words = [word for word in words if word in stop_words]
        counter.update(filtered_words)
    return counter

# Count stopwords in the dataset
stopword_counts = count_stopwords(dataset)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Get the most frequent stopwords
most_frequent_stopwords = [word for word, count in stopword_counts.most_common()]

# Calculate the cutoff for the top 50%
cutoff_index = len(most_frequent_stopwords) // 2
stopwords_to_remove = set(most_frequent_stopwords[:cutoff_index])

print("Stopwords to remove:", stopwords_to_remove)


Stopwords to remove: {'only', 'too', 'all', 'or', 'a', 'was', 'so', 'why', 'to', 'her', 'i', 'at', 'is', 'they', 'who', 'there', 'as', 'again', 'then', 'do', 'it', 'you', 'them', 'were', 'this', 'are', 'and', 'been', 'through', 'because', 'his', 'over', 'he', 'more', 'the', 'did', 'not', 'what', 'we', 'now', 'here', 'out', 'when', 'an', 'him', 'just', 'for', 'from', 'on', 'your', 'have', 'off', 'before', 'my', 'be', 'with', 'of', 'up', 'down', 'in', 'if', 'their', 'about', 'how', 'but', 'by', 'that', 'me', 'no', 'into', 'she', 'can', 'had', 'than'}


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the NLTK lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Remove unwanted characters and keep only certain punctuation
    text = re.sub(r"[^a-zA-Z0-9.,!?;:'\"() \-]", " ", text)
    # Convert to lower case
    text = text.lower()
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def remove_selected_stopwords_and_lemmatize(text, stopwords_to_remove):
    words = text.split()
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords_to_remove]
    return ' '.join(filtered_words)

def preprocess_data(examples):
    processed_texts = [clean_text(text) for text in examples['text']]
    processed_texts = [remove_selected_stopwords_and_lemmatize(text, stopwords_to_remove) for text in processed_texts]
    return tokenizer(processed_texts, truncation=True, padding=False)  # No padding here

# Apply preprocessing and tokenization
dataset = dataset.map(preprocess_data, batched=True)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Map:   0%|          | 0/5000000 [00:00<?, ? examples/s]

In [ ]:
# Assuming your dataset after map function is a Hugging Face dataset with proper structure
# This code will work if the dataset is structured with 'input_ids' and 'attention_mask' directly after processing

# Print the structure of the dataset to understand its composition
print("Dataset Structure: ", dataset.column_names)

# Accessing a sample data correctly
sample_data = dataset[0]  # Access the first entry in the dataset

# Print sample data
print("Sample Input IDs: ", sample_data['input_ids'])
print("Sample Attention Mask: ", sample_data['attention_mask'])

# Decode the tokens to see the actual text
decoded_text = tokenizer.decode(sample_data['input_ids'], skip_special_tokens=True)
print("Decoded Text: ", decoded_text)


Dataset Structure:  ['text', 'input_ids', 'attention_mask']
Sample Input IDs:  [23073, 837, 561, 24447, 1088, 2877, 2119, 837, 2712, 13373, 764]
Sample Attention Mask:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Decoded Text:  usually, would tearing around living room, playing toy.


In [ ]:
def add_input_target_pairs(example):
    # Shift input_ids to the right for target labels
    example['labels'] = example['input_ids'][1:] + [tokenizer.pad_token_id]
    return example

# Apply the function to add labels to the dataset
dataset = dataset.map(add_input_target_pairs)


Map:   0%|          | 0/5000000 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict

# Assuming your dataset is already loaded and named `dataset`
# Split the dataset into training and test sets initially
train_test_split = dataset.train_test_split(test_size=0.2)

# Split the test set further into validation and test sets
test_val_split = train_test_split['test'].train_test_split(test_size=0.5)

# Now assemble the final DatasetDict with train, validation, and test sets
final_dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': test_val_split['train'],
    'test': test_val_split['test']
})

# Print the size of each split to verify
print("Training set size:", len(final_dataset['train']))
print("Validation set size:", len(final_dataset['validation']))
print("Test set size:", len(final_dataset['test']))


Training set size: 4000000
Validation set size: 500000
Test set size: 500000


In [ ]:
# Define the base path for saving the datasets
base_path = "/content/datasets"

# Save each split to disk
final_dataset['train'].save_to_disk(f"{base_path}/train")
final_dataset['validation'].save_to_disk(f"{base_path}/validation")
final_dataset['test'].save_to_disk(f"{base_path}/test")

print("Datasets saved successfully to disk.")


Saving the dataset (0/2 shards):   0%|          | 0/4000000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500000 [00:00<?, ? examples/s]

Datasets saved successfully to disk.


In [ ]:
from datasets import load_from_disk

# Define the path where the train dataset is saved
train_dataset_path = "/content/datasets/train"

# Load the train dataset
train_dataset = load_from_disk(train_dataset_path)


In [ ]:
import torch
from transformers import GPT2Tokenizer

# Assuming you have already loaded the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Get a sample data point
sample_data = train_dataset[0]

# Decode the input and target IDs
decoded_input = tokenizer.decode(sample_data['input_ids'], skip_special_tokens=True)
decoded_target = tokenizer.decode(sample_data['labels'], skip_special_tokens=True)

# Print detailed information
print("Input IDs:", sample_data['input_ids'])
print("Target IDs:", sample_data['labels'])
print("Decoded Input:", decoded_input)
print("Decoded Target:", decoded_target)
print("Original Sentence:", sample_data['text'])  # Assuming 'text' field holds the original text


Input IDs: [2411, 2086, 35456, 6290, 64, 442, 392, 1754, 1913, 2356, 19551, 2961, 764]
Target IDs: [2086, 35456, 6290, 64, 442, 392, 1754, 1913, 2356, 19551, 2961, 764, 50256]
Decoded Input: relief flowed raina chandler strong pain ripped earlier.
Decoded Target: ief flowed raina chandler strong pain ripped earlier.
Original Sentence: relief flowed through raina chandler as strong as the pain had ripped through her earlier .


In [ ]:
from datasets import load_from_disk
import torch
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer

# Load the dataset
dataset_path = "/content/datasets/train"
dataset = load_from_disk(dataset_path)

# Check dataset structure
print("Dataset columns:", dataset.column_names)

# Example of accessing data
sample = dataset[0]
print("Sample Input IDs:", sample['input_ids'])
print("Sample Labels:", sample['labels'])


Dataset columns: ['text', 'input_ids', 'attention_mask', 'labels']
Sample Input IDs: [2411, 2086, 35456, 6290, 64, 442, 392, 1754, 1913, 2356, 19551, 2961, 764]
Sample Labels: [2086, 35456, 6290, 64, 442, 392, 1754, 1913, 2356, 19551, 2961, 764, 50256]


In [ ]:
def dynamic_padding_collate_fn(batch):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    max_length = tokenizer.model_max_length  # Typically 1024 for GPT-2

    # Calculate max length in this batch
    batch_max_length = min(max(len(x['input_ids']) for x in batch), max_length)

    # Prepare padded tensors
    padded_input_ids = torch.full((len(batch), batch_max_length), tokenizer.pad_token_id, dtype=torch.long)
    padded_labels = torch.full((len(batch), batch_max_length), -100, dtype=torch.long)  # Use -100 for ignored index

    for i, example in enumerate(batch):
        input_ids = example['input_ids']
        labels = example['labels']
        seq_length = min(len(input_ids), batch_max_length)

        padded_input_ids[i, :seq_length] = torch.tensor(input_ids[:seq_length], dtype=torch.long)
        padded_labels[i, :seq_length] = torch.tensor(labels[:seq_length], dtype=torch.long)

    attention_mask = (padded_input_ids != tokenizer.pad_token_id).int()

    return {
        'input_ids': padded_input_ids,
        'attention_mask': attention_mask,
        'labels': padded_labels
    }

# Create DataLoader with the custom collate function
train_loader = DataLoader(dataset, batch_size=4, collate_fn=dynamic_padding_collate_fn, shuffle=True)


In [ ]:
for batch in train_loader:
    print("Batch Input IDs:", batch['input_ids'].shape)
    print("Batch Labels:", batch['labels'].shape)
    break  # Only show the first batch for brevity


Batch Input IDs: torch.Size([4, 10])
Batch Labels: torch.Size([4, 10])


In [ ]:
from transformers import GPT2Tokenizer

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Assuming train_loader is already created and configured
# Example debugging step to decode and print batch content
for batch in train_loader:
    input_ids = batch['input_ids']
    labels = batch['labels']

    for i in range(min(len(input_ids), 5)):  # Limit to first 5 for clarity
        try:
            decoded_inputs = tokenizer.decode(input_ids[i], skip_special_tokens=True)
            decoded_labels = tokenizer.decode(labels[i].tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=True)
        except TypeError:
            continue

        print(f"Sample {i+1}:")
        print("Input IDs:", input_ids[i])
        print("Decoded Inputs:", decoded_inputs)
        print("Labels:", labels[i])
        print("Decoded Labels:", decoded_labels)
    break



Sample 4:
Input IDs: tensor([ 2093,   276,   837, 10342,  2119,  1566,  1043,   474,   897,  3375,
          582,  8018,  2583,  2057,  3127, 31557,   905,   764])
Decoded Inputs: asked, searching room until found jax talking man recognized host food network tv show.
Labels: tensor([  276,   837, 10342,  2119,  1566,  1043,   474,   897,  3375,   582,
         8018,  2583,  2057,  3127, 31557,   905,   764, 50256])
Decoded Labels: ed, searching room until found jax talking man recognized host food network tv show.


In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import GPT2LMHeadModel, GPT2Tokenizer, get_scheduler
from torch.cuda.amp import GradScaler, autocast
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# DataLoader setup
# Assuming 'train_loader' and 'valid_loader' are defined
train_loader = DataLoader(dataset, batch_size=64, collate_fn=dynamic_padding_collate_fn, shuffle=True)

# Setup optimizer with PyTorch's AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

# Initialize TensorBoard writer
writer = SummaryWriter('runs/gpt2_fine_tuning')

# Initialize the gradient scaler for AMP
scaler = GradScaler()

# Number of training epochs and warmup steps
epochs = 1
num_warmup_steps = 500  # Adjust as needed
num_training_steps = epochs * len(train_loader)

# Setup learning rate scheduler with warmup
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

# Best model checkpointing
best_loss = float('inf')
best_model_path = '/content/model'

# Training loop with tqdm progress bar and AMP
model.train()
for epoch in range(epochs):
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

    for batch_idx, batch in enumerate(progress_bar):
        batch = {k: v.to(device) for k, v in batch.items()}

        with autocast():
            outputs = model(**batch)
            loss = outputs.loss

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        progress_bar.set_postfix(loss=loss.item())
        total_loss += loss.item()
        writer.add_scalar('Training loss', loss.item(), epoch * len(train_loader) + batch_idx)

    avg_loss = total_loss / len(train_loader)
    writer.add_scalar('Average Training Loss', avg_loss, epoch)

    # Save the best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        model.save_pretrained(best_model_path)
        tokenizer.save_pretrained(best_model_path)
        print("Saved new best model")

writer.close()

print("Training completed.")


Epoch 1/1:   0%|          | 0/62500 [00:00<?, ?it/s]

KeyboardInterrupt: 